In [9]:
from process_bigraph import Process, Composite
from biosimulators_processes import CORE  


fp = '/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/test_suite/examples/sbml-core/Elowitz-Nature-2000-Repressilator/BIOMD0000000012_url.xml'
fp2 = '/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/test_suite/examples/sbml-core/Beard2005_Mitochondrial_Respiration.xml'

doc = {
        'ode': {
            '_type': 'process',
            'address': f'local:copasi',
            'config': {
                'model_file': fp
            },
            'inputs': {
                'time': ['time_store'],
                'species_concentrations': ['species_concentrations_store']
            },
            'outputs': {
                'time': ['time_store'],
                'species_concentrations': ['species_concentrations_store'],
                'reaction_derivatives': ['reaction_derivatives_store']
            }
        },
        'fba': {
            '_type': 'process',
            'address': f'local:cobra',
            'config': {
                'model_file': fp,
                'objective': {
                    'domain': 'protein',
                    'name': 'LacI'
                }
            },
            'inputs': {
                'reaction_derivatives': ['reaction_derivatives_store']
            },
            'outputs': {
                'fluxes': ['fluxes_store']
            }
        },
        'emitter': {
            '_type': 'step',
            'address': 'local:ram-emitter',
            'config': {
                'emit': {
                    'time': 'float',
                    'floating_species_concentrations': 'tree[float]',
                    'reaction_derivatives': 'tree[float]',
                    'fluxes': 'tree[float]'
                }
            },
            'inputs': {
                'time': ['time_store'],
                'species_concentrations': ['species_concentrations_store'],
                'reaction_derivatives': ['reaction_derivatives_store'],
                'fluxes': ['fluxes_store']
            }
        }
    }

Cannot register SimulariumSmoldynStep. Error:
**
No module named 'simulariumio'
**
Cannot register MongoDatabaseEmitter. Error:
**
No module named 'simulariumio'
**


In [2]:
composition = Composite(config={'state': doc}, core=CORE)

No objective coefficients in model. Unclear what should be optimized


In [3]:
composition.run(20)
composition.gather_results()

{('emitter',): [{'time': 0.0,
   'species_concentrations': {'LacI protein': 0.0,
    'TetR protein': 0.0,
    'cI protein': 0.0,
    'LacI mRNA': 0.0,
    'TetR mRNA': 20.0,
    'cI mRNA': 0.0},
   'reaction_derivatives': {'degradation of LacI transcripts': 0.0,
    'degradation of TetR transcripts': 6.931471805599453,
    'degradation of CI transcripts': 0.0,
    'translation of LacI': 0.0,
    'translation of TetR': 138.62943611198907,
    'translation of CI': 0.0,
    'degradation of LacI': 0.0,
    'degradation of TetR': 6.931471805599453,
    'degradation of CI': 0.0,
    'transcription of LacI': 30.0,
    'transcription of TetR': 30.0,
    'transcription of CI': 30.0},
   'fluxes': {'degradation of LacI transcripts': 0.0,
    'degradation of TetR transcripts': 0.0,
    'degradation of CI transcripts': 1000.0,
    'translation of LacI': 0.0,
    'translation of TetR': 0.0,
    'translation of CI': 0.0,
    'degradation of LacI': 0.0,
    'degradation of TetR': 0.0,
    'degradatio